# Monte Carlo Tree Search algorithm

First we import the necessary libraries needed for the search

In [1]:
import math
import torch
import numpy as np

In [13]:
import model

ImportError: cannot import name 'game' from 'lib' (unknown location)

In [12]:
import game

## MCTS functions:
1- Initialization:  
    > c_pcut: used to increase exploration*

In [ ]:
class MCTS:
    """
    MonteCarlo Search Tree(MCTS): 
    """
    def __init__(self, c_puct=1.0):
        self.c_puct = c_puct
        # visi count: -> [N(s,a)]
        self.visit_count = {}
        # value: -> [W(s,a)]
        self.value = {}
        # value average: -> [Q(s,a)]
        self.value_avg = {}
        # prior probability -> [P(s,a)]
        self.prob = {}
    
    def __len__(self):
        """Return the object length """
        return len(self.value)
    
    def is_leaf(self, state_int):
        """
        Checks whether the state has been explored
        before or not. i.e. leaf node
        """
        return state_int not in self.prob
    
    def find_leaf(self,state_int, player):
        """
        Description: finds a leaf state and return its value, state_int, player and the states and actions that lead to 
            the leaf state.
        
        Action with highest score is selected from a list of possible actions at each state
        A random noise is added to score to imporve exploration
        It returns value of -1.0 if currrent player wins, or 0.0 in the case of a draw. Otherwise value remains None
        """
        states = []
        actions = []
        cur_state = state_int
        cur_player = player
        value = None
        while not self.is_leaf(cur_state):
            states.append(cur_state)
            
            # Gather statistics about the current node in order to decicde which action to take
            counts = self.count[cur_state]
            total_counts = math.sqrt(sum(count))
            value_avg = self.value_avg[cur_state]
            probs = self.prob[cur_state]
            
            # add some noise to starting point to imporve exploration
            if cur_state == state_int:
                noises = np.random.dirichlet([0.03]*game.GAME_COLS)
                probs = [0.75 * prob + 0.25 * noise for prob,noise in zip(probs,noises)]
            
            # Action:
            score = [value + self.c_puct * prob * total_counts/(1+count) for value,prob,count in zip(value_avg,probs,counts)]
            # Insure invalid actions are not selected
            invalid_actions = set(range(game.GAME_COLS)) - set(game.possible_moves(cur_state))
            for invalid in invalid_actions:
                score[invalid] = -np.inf
            # select action with highest score
            action = int(np.argmax(score))
            # play the action
            cur_state, won = game.move(cur_state, action, cur_player)
            # switch player from 0 -> 1 or vice versa
            cur_player = 1 - cur_player
            # Check if someone won
            if won is not None:
                # we set the value to -1.0 because we switched the player in the previous step
                value = -1.0
            # if there is a draw
            if value is None and len(game.possible_moves(cur_state))==0:
                value = 0.0
                
        return value, cur_state, cur_player, states, actions
    
    def search_minibatch(self,batch_size,state_int,player,net,device='cpu'):
        """
        Find a batch of leafs and explode them
        """
        backup_queue = []
        expand_states = []
        expand_players = []
        expand_queue = []
        planned = set()
        
        for _ in range(batch_size):
            value, leaf_state, leaf_player, states, actions = self.find_leaf(state_int, player)
            if value is not None:
                backup_queue.append((value, states, actions))
            else:
                if leaf_state not in planned:
                    planned.add(leaf_state)
                    leaf_state_list = game.decode_binary(leaf_state)
                    expand_states.append(leaf_state_list)
                    expand_players.append(leaf_player)
                    expand_queue.append((leaf_state, states, actions))
        
        # Expand the new leafs if there is any
        if expand_queue:
            batch = model.
        
    

In [ ]:
mcts = MCTS()
state_int = game.INITIAL_STATE


In [ ]:
mcts.find_leaf(state_int, 1)